In [46]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist

%matplotlib inline
from matplotlib import lines as mlines
import matplotlib.pyplot as plt
from cycler import cycler

In [47]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.1
lumis['2018'] = 59.9

nfiles_mc = {}
nfiles_mc['2016'] = 64
nfiles_mc['2017'] = 89
nfiles_mc['2018'] = 106

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [48]:
year = '2018'
outsum = processor.dict_accumulator()

In [49]:
# Load all MC
for n in range(1,nfiles_mc[year]+1):
    print(n)
    filename = 'condor/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['cutflow'].scale(scale_lumi, 'dataset')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106


In [50]:
templates = outsum['cutflow'].group('dataset', hist.Cat('process', 'Process'), pmap)

/uscms/home/jennetd/.local/lib/python3.6/site-packages/coffea/hist/hist_tools.py:346: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f927cc185c0>
  warnings.warn("Not all requested indices present in %r" % self, RuntimeWarning)


In [51]:
#pmap = {}
#pmap['otherH'] = ['ttH','WH','ZH','VBF']
#pmap['ggF'] = ['ggF']
#pmap['VV'] = ['VV']
#pmap['top'] = ['ttbar','singlet']
#pmap['Zjets'] = ['Zjets']
#pmap['Wjets'] = ['Wjets']
#pmap['QCD'] = ['QCD']

cutflow = templates.sum('genflavor').integrate('region','signal')

In [52]:
cutflow.values()

{('ZH',): array([228.17614837, 228.17614837, 123.60794883, 114.38166405,
        114.38166405,  68.72981293,  33.08095507,  23.63712468,
         18.10985458,   6.73228906,   0.        ]),
 ('WH',): array([231.1518157 , 231.1518157 , 135.54101663, 126.68822406,
        126.68822406,  78.99048743,  32.59748044,  30.21184791,
         22.62380834,   8.07146633,   0.        ]),
 ('ttH',): array([968.85262986, 968.85262986, 628.72547509, 524.60738898,
        524.60738898, 186.52347452,  55.56558242,  41.8966008 ,
         16.17302956,   4.08260178,   0.        ]),
 ('VBF',): array([369.10938335, 369.10938335, 134.8963816 , 125.9111166 ,
        125.9111166 ,  71.947766  ,  58.15957941,  56.20830346,
         45.86749462,  27.43118321,   0.        ]),
 ('ggF',): array([897.10341568, 897.10341568, 374.70891096, 345.37823357,
        345.37823357, 175.44640549, 133.00924962, 129.51608951,
        102.19545863,  60.05142588,   0.        ]),
 ('QCD',): array([1.15516711e+08, 1.15516711e+08, 3.

In [53]:
import pandas as pd

In [54]:
df1 = pd.DataFrame([])

df1['ggF'] = cutflow.values()[('ggF',)]
df1['VBF'] = cutflow.values()[('VBF',)]
df1['WH'] = cutflow.values()[('WH',)]
df1['ZH'] = cutflow.values()[('ZH',)]
df1['ttH'] = cutflow.values()[('ttH',)]

df1 = df1[3:-1].astype('int')
df1.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','DeepDoubleB']

df1.to_latex(buf=year+'/cutflow-sig.tex')

In [55]:
df2 = pd.DataFrame([])

df2['QCD'] = cutflow.values()[('QCD',)]
df2['Wjets'] = cutflow.values()[('Wjets',)]
df2['Zjets'] = cutflow.values()[('Zjets',)]
df2['VV'] = cutflow.values()[('VV',)]
df2['ttbar'] = cutflow.values()[('ttbar',)]
df2['singlet'] = cutflow.values()[('singlet',)]

df2 = df2[3:-1].astype('int')
df2.index = ['Jet kinematics','Jet ID','Jet $N_2^\text{DDT}$','Opp. hem. b veto','MET $<$ 140 GeV','No leptons','DeepDoubleB']

df2.to_latex(buf=year+'/cutflow-bkg.tex')